In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, AzureCliCredential

In [2]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /home/daniel/repos/aml/config.json


In [3]:
[print(x) for x in ml_client.data.list()]

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: ERROR: AADSTS70043: The refresh token has expired or is invalid due to sign-in frequency checks by conditional access. The token was issued on 2025-01-06T07:49:45.5590291Z and the maximum allowed lifetime for this request is 14400. Trace ID: 7a0c140c-ad9a-49bb-9db8-4f1897f77d00 Correlation ID: cec5b62f-e84d-4025-b7eb-3f5eb34990a7 Timestamp: 2025-01-07 09:49:22Z
Interactive authentication is needed. Please

ClientAuthenticationError: DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: ERROR: AADSTS70043: The refresh token has expired or is invalid due to sign-in frequency checks by conditional access. The token was issued on 2025-01-06T07:49:45.5590291Z and the maximum allowed lifetime for this request is 14400. Trace ID: 7a0c140c-ad9a-49bb-9db8-4f1897f77d00 Correlation ID: cec5b62f-e84d-4025-b7eb-3f5eb34990a7 Timestamp: 2025-01-07 09:49:22Z
Interactive authe

In [5]:

%%bash
mkdir azureml-environment
echo """
FROM pytorch/pytorch:2.0.0-cuda11.7-cudnn8-runtime

# Downloads to user config dir
ADD https://ultralytics.com/assets/Arial.ttf https://ultralytics.com/assets/Arial.Unicode.ttf /root/.config/Ultralytics/

# Install linux packages
ENV DEBIAN_FRONTEND noninteractive
RUN apt update
RUN TZ=Etc/UTC apt install -y tzdata
RUN apt install --no-install-recommends -y gcc git zip curl htop libgl1-mesa-glx libglib2.0-0 libpython3-dev gnupg g++

# Security updates
# https://security.snyk.io/vuln/SNYK-UBUNTU1804-OPENSSL-3314796
RUN apt upgrade --no-install-recommends -y openssl tar

RUN pip install ultralytics==8.0.132
RUN pip install azureml-mlflow==1.52.0
RUN pip install mlflow==2.4.2
""" > azureml-environment/Dockerfile

mkdir: cannot create directory ‘azureml-environment’: File exists


In [11]:
#lets create a compute instance
from azure.ai.ml.entities import ComputeInstance, AmlCompute, Environment, BuildContext


ci_basic_name = "defaultclustersbxdondorp"
ci_basic = AmlCompute(name=ci_basic_name, size="STANDARD_DS3_v2")
# ml_client.begin_create_or_update(ci_basic).result()

In [12]:
env = Environment(name="dockerenv", build=BuildContext(path="azureml-environment"))
ml_client.environments.create_or_update(env)

Uploading azureml-environment (0.0 MBs): 100%|██████████| 684/684 [00:00<00:00, 24850.61it/s]




Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'dockerenv', 'description': None, 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/11f51dee-57cd-4d47-b542-8e244706e163/resourceGroups/sbx-dondorp/providers/Microsoft.MachineLearningServices/workspaces/amlsbxdondorp/environments/dockerenv/versions/1', 'Resource__source_path': '', 'base_path': '/home/daniel/repos/aml/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f90a4335ea0>, 'serialize': <msrest.serialization.Serializer object at 0x7f90a4335cf0>, 'version': '1', 'conda_file': None, 'build': <azure.ai.ml.entities._assets.environment.BuildContext object at 0x7f90a4335660>, 'inference_config': None, 'os_type': 'Linux', 'conda_file_path': None, 'path': None, 'datastore': None, 'upload_h

In [114]:
envs = ml_client.environments.list()
envs = list(envs)
print(envs)
if "testenv" not in [x.name for x in envs]:
    env = Environment(image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
                      conda_file="/home/daniel/repos/aml/environments/basic_env.yml",
                      name="testenv",
                      description="test environment from conda file",
                      )
    ml_client.environments.create_or_update(env)

[Environment({'arm_type': 'environment_version', 'latest_version': '2', 'image': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': True, 'auto_delete_setting': None, 'name': 'testenv2', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/11f51dee-57cd-4d47-b542-8e244706e163/resourceGroups/sbx-dondorp/providers/Microsoft.MachineLearningServices/workspaces/amlsbxdondorp/environments/testenv2', 'Resource__source_path': '', 'base_path': '/home/daniel/repos/aml/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7efd10547070>, 'serialize': <msrest.serialization.Serializer object at 0x7efd28957eb0>, 'version': None, 'conda_file': None, 'build': None, 'inference_config': None, 'os_type': None, 'conda_file_path': None, 'path': None, 'datastore': None, 'upload_hash': None, 'translated_conda_file': None}), Environment({'arm_type': 'environment_version', 'latest_version': '1', 'i

In [ ]:
custom_env_name = "aml-scikit-learn"

custom_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for Credit Card Defaults job",
    tags={"scikit-learn": "1.0.2"},
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
custom_job_env = ml_client.environments.create_or_update(custom_job_env)

In [26]:
from azure.ai.ml import command


# Define the job
job = command(
    code="/home/daniel/repos/aml/notebooks/scripts",
    command="python hello_world.py",
    environment="azureml:dockerenv:1",  # Use the registered environment
    compute="defaultcomputesbxdondorp",
    display_name="HelloWorld",
    experiment_name="Hello World Experiment",
)


In [27]:
print(job)

type: command
environment: azureml:dockerenv:1
component:
  name: azureml_anonymous
  version: '1'
  display_name: HelloWorld
  type: command
  command: python hello_world.py
  environment: azureml:dockerenv:1
  code: /home/daniel/repos/aml/notebooks/scripts
  is_deterministic: true
compute: azureml:defaultcomputesbxdondorp



In [31]:
%%bash
# For this tutorial we will use the coco128 dataset: https://www.kaggle.com/ultralytics/coco128. We will create an AzureML data asset to bookmark our dataset and easily use the dataset for various trainings.
# wget https://ultralytics.com/assets/coco128.zip
unzip coco128.zip

Archive:  coco128.zip
   creating: coco128/
  inflating: coco128/LICENSE         
   creating: coco128/images/
   creating: coco128/images/train2017/
  inflating: coco128/images/train2017/000000000612.jpg  
  inflating: coco128/images/train2017/000000000404.jpg  
  inflating: coco128/images/train2017/000000000438.jpg  
  inflating: coco128/images/train2017/000000000389.jpg  
  inflating: coco128/images/train2017/000000000564.jpg  
  inflating: coco128/images/train2017/000000000149.jpg  
  inflating: coco128/images/train2017/000000000605.jpg  
  inflating: coco128/images/train2017/000000000349.jpg  
  inflating: coco128/images/train2017/000000000201.jpg  
  inflating: coco128/images/train2017/000000000599.jpg  
  inflating: coco128/images/train2017/000000000572.jpg  
  inflating: coco128/images/train2017/000000000360.jpg  
  inflating: coco128/images/train2017/000000000370.jpg  
  inflating: coco128/images/train2017/000000000562.jpg  
  inflating: coco128/images/train2017/000000000589.j

In [32]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# Create AzureML dataset

my_data = Data(
    path="coco128",
    type=AssetTypes.URI_FOLDER,
    description="Coco 128 dataset",
    name="coco128"
)

ml_client.data.create_or_update(my_data)

Uploading coco128 (7.0 MBs): 100%|██████████| 7003402/7003402 [00:00<00:00, 7544360.34it/s] 




Data({'path': 'azureml://subscriptions/11f51dee-57cd-4d47-b542-8e244706e163/resourcegroups/sbx-dondorp/workspaces/amlsbxdondorp/datastores/workspaceblobstore/paths/LocalUpload/4cf631dbb6578745b50792fa18bee43a/coco128/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'coco128', 'description': 'Coco 128 dataset', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/11f51dee-57cd-4d47-b542-8e244706e163/resourceGroups/sbx-dondorp/providers/Microsoft.MachineLearningServices/workspaces/amlsbxdondorp/data/coco128/versions/1', 'Resource__source_path': '', 'base_path': '/home/daniel/repos/aml/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f908c24c220>, 'serialize': <msrest.serialization.Serializer object at 0x7f908c24fa00>, 'version': '1', 'latest_version': None, 'datastore': None})

In [98]:
from azure.ai.ml import command
from azure.ai.ml import Input

job = command(
    inputs=dict(
        training_data=Input(
            type="uri_folder",
            path="azureml:coco128:1",
        ),
    #     # model_to_train=Input(
    #     #     type="custom_model",
    #     #     path="azureml:yolov8n:1"
    #     # )
    ),
    code="scripts",
    command="""
        python3 hello_world.py
    """,
    environment="azureml:dockerenv:1",
    compute="defaultclustersbxdondorp",
    display_name="hello_world_without_inputs",
    experiment_name="hello_world_no_inputs_at_all"
)

In [96]:
ml_client.create_or_update(job)

HttpResponseError: (UserError) Unable to create or update run boring_jewel_dfkm8z97tf : Bad Request.
Code: UserError
Message: Unable to create or update run boring_jewel_dfkm8z97tf : Bad Request.
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "92db341351e1d860bc60030690f9010b",
        "request": "a39a540914d60abd"
    }
}Type: Environment
Info: {
    "value": "westeurope"
}Type: Location
Info: {
    "value": "westeurope"
}Type: Time
Info: {
    "value": "2025-01-06T10:04:38.8246202+00:00"
}Type: InnerError
Info: {
    "value": {
        "code": "BadArgument",
        "innerError": {
            "code": "CreatRunBadRequest",
            "innerError": null
        }
    }
}